In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
train_df = pd.read_parquet('/kaggle/input/sibalfahack/Siberian Alfa Hack Materials/Siberian Alfa Hack Materials/train.parquet')
test_df = pd.read_parquet('/kaggle/input/sibalfahack/Siberian Alfa Hack Materials/Siberian Alfa Hack Materials/test.parquet')
df = pd.concat([train_df, test_df], ignore_index=True)

In [ ]:
train_df.shape, test_df.shape

# Exploratory Data Analysis

In [ ]:
class DataProcessor:
    def __init__(self, data, test_data, target, list_drop_cols):
        self.data = data
        self.test_data = test_data
        self.cat_cols = list(data.select_dtypes(include=['object']).columns) + list(data.select_dtypes(include=['category']).columns)
        self.target = target
        self.list_drop_cols = list_drop_cols
        
    def rm_spare_cols(self):
        """
        Удаление лишних столбцов.
        """
        self.data.drop(self.list_drop_cols, axis=1, inplace=True)
        
    def rm_NaN_treshold(self, treshold = 200000):
        """
        Удаление всех столбцов, NAN в которых больше заданного порога.
        """
        self.data = self.data.columns[self.data.isnull().sum() > treshold]
        
    def rm_cols_small_target(self, threshold=100):
        columns_to_check = self.data.columns.difference([self.target])
        selected_columns = []
        for column in columns_to_check:
            df_subset = pd.DataFrame({column: self.data[column], target_column: self.data[self.target]})
            df_subset = df_subset.dropna()
            count_ones = df_subset[self.target].sum()
            if count_ones < threshold:
                selected_columns.append(column)

        return self.data.drop(columns = selected_columns, inplace = true)
        
    def rm_or_fill_values(self, rm = True):
        """
        Удаление строк с отрицательными значениями или замена ячеек на NaN.
        """
        if self.cat_cols is not None:
            if rm:
                self.data[~self.cat_cols] = self.data[~self.cat_cols][self.data[~self.cat_cols].le(0).all(axis=1)]
            else:
                self.data[~self.cat_cols].mask(self.data < 0, 0, inplace = True)
        
    def preprocess_data(self, rm = True, rmnan_treshold = 200000, treshold = 100):
        """
        Метод для предобработки данных
        """
        self.rm_spare_cols()
        self.rm_or_fill_values(rm = True)
        self.data[self.cat_cols] = self.data[self.cat_cols].astype("category")
        self.rm_NaN_treshold(rmnan_treshold)
        self.rm_cols_small_target()
        
    def visualize_all_columns_distribution(self, cols_list, labels = self.target):
        """
        Визуализация списка столбцов по категориям
        """
        import matplotlib as plt
        for column in cols_list:
            df_subset = pd.DataFrame({column: self.data[column], target_column: self.data[labels]})
            df_subset = df_subset.dropna()
            count_ones = df_subset[target_column].sum()
            plt.figure(figsize=(10, 6))
            sns.histplot(data=df_subset, x=column, hue=target_column, bins=30, kde=True)
            plt.title(f'Distribution of "{column}" (Total Label: {count_ones} ones)')
            plt.xlabel(column)
            plt.ylabel('Count')
            plt.legend(title=target_column)
            plt.show()
        
    def encode_categorical_features(self):
        # Кодирует категориальные признаки числовыми значениями
        for col in self.cat_cols:
            le = LabelEncoder()
            self.data[col] = le.fit_transform(self.data[col])

    def clean_data(self):
        # Метод для очистки данных
        # удаление дубликатов, обработка пропущенных значений
        self.data = self.data.drop_duplicates()
        # self.data = self.data.dropna()

    def clean_and_preprocess(self):
        # Метод для последовательной очистки и предобработки данных
        self.clean_data()
        self.preprocess_data()

    def visualize_feature(self, col):
        # Метод для визуализации признака
        if col in self.cat_cols or col == 'total_target':
            fig = px.pie(self.data, names=col, title=f'Распределение признака {col}')
        else:
            fig = px.histogram(self.data, x=col, title=f'Распределение признака {col}', labels={col: col})
        fig.show()
    
    def visualize_target(self, target):
        # Метод для визуализации распределения таргетов
        fig = px.histogram(self.data, x=target, color=target,
                           title='Распределение таргета', labels={target: target},
                           category_orders={target: [0, 1]}, barmode='overlay')
        fig.update_layout(showlegend=False)
        fig.show()
    
    def visualize_correlations(self, max_corr, min_corr):
        # Метод для визуализации скоррелированных фичей
        labels_to_drop = {(self.data.columns[i], self.data.columns[j]) for i in range(self.data.shape[1]) for j in range(i + 1)}
        au_corr = self.data.corr().unstack()
        au_corr = au_corr.drop(labels=labels_to_drop).sort_values(ascending=False)
        filtercorr = au_corr[((au_corr >= min_corr) & (au_corr <= max_corr)) | ((au_corr <= -min_corr) & (au_corr >= -max_corr)) & (au_corr !=1.0)]
        au_corr = filtercorr.unstack(level=0)
        fig = px.imshow(au_corr, aspect="auto")
        fig.update_layout(font=dict(size=8))
        fig.show()
        
    def get_churn_category(self, group_by_column, target_column):
        # Метод для расчета процента оттока по категориям для категориальных фичей
        grouped_data = self.data.groupby(group_by_column, as_index=False).agg({target_column: ['sum', 'count']})
        grouped_data.columns = [group_by_column, 'Churn_Sum', 'Churn_Count']
        grouped_data['Churn_Percentage'] = 100 * grouped_data['Churn_Sum'] / grouped_data['Churn_Count']
        grouped_data = grouped_data.sort_values('Churn_Percentage').reset_index(drop=True)
        return grouped_data

In [ ]:
data_processor = DataProcessor(train_df, 'total_target')

In [ ]:
data_processor.visualize_target('total_target')

In [ ]:
data_processor.visualize_feature('segment')

In [ ]:
data_processor.visualize_feature('sum_a_oper_1m')

In [ ]:
data_processor.visualize_correlations(min_corr=0.7, max_corr=0.9999)

In [ ]:
data_processor.get_churn_category(group_by_column='segment', target_column='target_2')